# 第一版存取

In [6]:
import pymongo
from pymongo import MongoClient
import datetime
import json
import time
from functools import reduce

#連Mongodb
client = MongoClient('172.20.26.39', 27017)
#連DB名
db=client['rawData']
#秀DB內的collection list
db.collection_names()

['wowprimeipeenAddCommentURL', 'wowprimeipeen']

In [ ]:
#選擇我們想要的collection
collection = db.wowprimeipeen
#list(dict)的去重公式
f = lambda x,y:x if y in x else x + [y]

#不斷重複動作
while True:
    #讀取資料
    try:
        with open("./data/diendata.json","r") as d:
            diendata=json.load(d)
            print("讀取完畢")
            #做去重動作
            diendata = reduce(f, [[], ] + diendata)
            print("去重完畢")
            with open("./data/mongodiendata.json","w") as d:
                json.dump(diendata,d)
            print("另存完畢")
            #比數變多，則做儲存動作
            if len(diendata)>len(list(collection.find({}))):
                collection.drop()
                collection.insert_many(diendata)
                print("存資料庫完畢")
                print("目前有%s筆資料"%len(list(collection.find({}))))
                #將去重的資料再儲存於外部json檔
                
            #如果資料筆數不變，則停止繼續儲存的行為
            else:
                print("已存最大筆數")
                break
    except Exception as e:
        print(e)
        time.sleep(3)
        continue
    time.sleep(900)

讀取完畢
去重完畢
另存完畢
存資料庫完畢
目前有159156筆資料


{'Ncomment': 0,
 'address': 0,
 'averagecost': 0,
 'bigadd': 0,
 'bigstyle': '網購包裝食品',
 'collecount': 0,
 'id': 1002616,
 'ipeenscore': 0,
 'lat': 0,
 'lng': 0,
 'name': 'Hotea來找茶',
 'othername': 0,
 'reviewdate': '2015/02/04',
 'smalladd': 0,
 'smallstyle': '其他網購包裝食品',
 'status': '正常營業',
 'tele': 0,
 'viewcount': 251}

In [22]:
#將資料讀出存成excel

import pandas as pd
import re

collection = db.wowprimeipeen
data=list(collection.find({}))

print("讀取完畢")

#加入一些資料
for i in data:
    try:
        i['othername']=re.findall("\((\w*[店|館|門市|百貨|街|夜市|商場|時代|廣場|中心|三越|大學|中|區|高鐵])\)",i['name'])[0]
    except:
        i['othername']=0
    if i['othername']!=0:
        i['name']=i['name'].split("(")[0]
    try:
        i["bigadd"]=re.findall("(..[市|縣])(\w\w?\w?[區|市|鎮|鄉])",
                               i["address"])[0][0]
#         i["bigadd"]=re.findall("(..[市|縣])(.+[區|市|鎮|鄉]).+[路|街|地段|巷][0-9]+[-|之]?[0-9]*?[號|巷|弄]?[0-9]*[號|巷|弄|樓]",
#                                i["address"])[0][0]
    except:
        i['bigadd']=0
    try:
        i["smalladd"]=re.findall("(..[市|縣])(\w\w?\w?[區|市|鎮|鄉])",
                           i["address"])[0][1]
    except:
        i["smalladd"]=0
#     if i["address"]==0 and i["status"]!="結束營業":
#         i["status"]="無地址可能已搬遷"
print("資料編輯完畢")
        
# Create a Pandas dataframe from some data.
df = pd.DataFrame(data)
df

df=df.reindex(columns=['id','name','othername','tele','address','bigadd',"smalladd",'lat','lng','bigstyle','smallstyle','averagecost',
                    'ipeenscore','status','Ncomment','reviewdate'])
# # Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('./data/pandas_simple.xlsx', engine='xlsxwriter')

# # Convert the dataframe to an XlsxWriter Excel object.
df.to_excel(writer, sheet_name='Sheet1')

# # Close the Pandas Excel writer and output the Excel file.
writer.save()
print("儲存完畢")

讀取完畢
資料編輯完畢
儲存完畢


In [3]:
# #存進mongo一次用
# collection = db.wowprimeipeen
# f = lambda x,y:x if y in x else x + [y]
# with open("./data/diendata.json","r") as d:
#     diendata=json.load(d)
#     print("讀取完畢")
#     #做去重動作
#     diendata = reduce(f, [[], ] + diendata)
#     print("去重完畢")
#     with open("./data/mongodiendata.json","w") as d:
#             json.dump(diendata,d)
#     print("另存完畢")
#     #比數變多，則做儲存動作
#     if len(diendata)>len(list(collection.find({}))):
#         collection.drop()
#         collection.insert_many(diendata)
#         print("存資料庫完畢")
#         print("目前有%s筆資料"%len(list(collection.find({}))))

讀取完畢
去重完畢
另存完畢
存資料庫完畢
目前有161821筆資料


# 用mongodb的資料，修復原始json檔

In [7]:
x=list(collection.find({}))
[i.pop("_id") for i in x]

with open("./data/diendata.json","w") as d:
    json.dump(x,d)

KeyboardInterrupt: 

In [20]:
# # 用mongodb的資料，修復原始json檔 修復地址
# from bs4 import BeautifulSoup as bs
# collection = db.wowprimeipeen
# x=list(collection.find({}))
# [i.pop("_id") for i in x]
# for i in x:
#     if i['address']==0 and i['lng']!=0:
#         try:
#             with open("./data/html2/"+str(i['id'])+".html","r",encoding='utf8') as f:
#                 dienhtml=f.read()
#                 soup=bs(dienhtml,"lxml")
#                 info=soup.select_one("#shop-header > div.info")
#                 i["address"]=info.find('a', attrs={'data-label': '上方地址'}).text.replace("\n"," ").strip()
# #                 print(i["address"])
#         except Exception as e:
#             print(e)

# # 用mongodb的資料，修復原始json檔 修復狀態
# from bs4 import BeautifulSoup as bs
# collection = db.wowprimeipeen
# x=list(collection.find({}))
# [i.pop("_id") for i in x]
# n=0
# for i in x:
#     if i['address']==0 and i['status']=="正常營業":
#         try:
#             with open("./data/html2/"+str(i['id'])+".html","r",encoding='utf8') as f:
#                 n+=1
#                 if n%50==0:
#                     print(n)
#                 dienhtml=f.read()
#                 soup=bs(dienhtml,"lxml")
#                 info=soup.select_one("#shop-header > div.info")
#                 if "已搬遷" in info.select_one("h1").text:
#                     i["status"]="已搬遷"
                
#         except Exception as e:
#             print(e)
            
            
# with open("./data/diendata.json","w") as d:
#     json.dump(x,d)


50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
[Errno 2] No such file or directory: './data/html2/26358.html'
800
850
900
950
1000
[Errno 2] No such file or directory: './data/html2/79200.html'
1050
1100
1150
[Errno 2] No such file or directory: './data/html2/589696.html'
1200
[Errno 2] No such file or directory: './data/html2/66267.html'
1250
[Errno 2] No such file or directory: './data/html2/119564.html'
1300
1350
[Errno 2] No such file or directory: './data/html2/117702.html'
1400
1450
1500
1550
[Errno 2] No such file or directory: './data/html2/69117.html'
1600
1650
1700
[Errno 2] No such file or directory: './data/html2/118813.html'
1750
[Errno 2] No such file or directory: './data/html2/38312.html'
1800
1850
[Errno 2] No such file or directory: './data/html2/75277.html'
1900
1950
2000
2050
2100
2150
2200
[Errno 2] No such file or directory: './data/html2/617588.html'
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
[Errno 2] No such file or directory: 

In [21]:
collection.drop()
collection.insert_many(x)
[i.pop("_id") for i in x]
with open("./data/mongodiendata.json","w") as d:
    json.dump(x,d)
with open("./data/diendata.json","w") as d:
    json.dump(x,d)

In [18]:
x=list(collection.find({}))
len(x)

56176

In [17]:
with open("./data/diendata.json","r") as d:
    diendata=json.load(d)
len(diendata)

59828

# 第二版存取(含CommentURL)

In [1]:
import pymongo
from pymongo import MongoClient
import datetime
import json
import time
from functools import reduce

#連Mongodb
client = MongoClient('172.20.26.39', 27017)
#連DB名
db=client['rawData']
#秀DB內的collection list
db.collection_names()


['wowprimeipeenAddCommentURL', 'wowprimeipeen']

In [2]:
#選擇我們想要的collection(亦可直接新增)
collection = db.wowprimeipeenAddCommentURL

In [ ]:
#list(dict)的去重公式
f = lambda x,y:x if y in x else x + [y]

loadPath='./data/diendataAddCommentUrl.json'
#不斷重複動作
while True:
    #讀取資料
    try:
        with open(loadPath,"r") as d:
            diendata=json.load(d)
            print("讀取完畢")
            #做去重動作
            diendata = reduce(f, [[], ] + diendata)
            print("去重完畢")
            with open("./data/mongodiendata2.json","w") as d:
                    json.dump(diendata,d)
            print("另存完畢")
            #比數變多，則做儲存動作
            if len(diendata)>len(list(collection.find({}))):
                collection.drop()
                collection.insert_many(diendata)
                print("存資料庫完畢")
                print("目前有%s筆資料"%len(list(collection.find({}))))
                #將去重的資料再儲存於外部json檔
                
            #如果資料筆數不變，則停止繼續儲存的行為
            else:
                print("已存最大筆數")
                break
    except Exception as e:
        print(e)
        time.sleep(3)
        continue
    time.sleep(900)

讀取完畢
去重完畢
另存完畢
存資料庫完畢
目前有16464筆資料


In [4]:
x=list(collection.find({}))
[i.pop("_id") for i in x]
x

[{'Ncomment': 0,
  'address': '台東縣台東市安慶街16號',
  'averagecost': 0,
  'bigstyle': '咖啡、簡餐、茶',
  'collecount': 0,
  'comments': [],
  'id': 10,
  'ipeenscore': 0,
  'lat': 22.749409424335585,
  'lng': 121.14442065867614,
  'name': '蔡文益',
  'reviewdate': '2006/12/13',
  'smallstyle': '咖啡專賣',
  'status': '正常營業',
  'tele': '089325593',
  'url': 'http://www.ipeen.com.tw/shop/10',
  'viewcount': 5391},
 {'Ncomment': 0,
  'address': 0,
  'averagecost': 0,
  'bigstyle': '咖啡、簡餐、茶',
  'collecount': 1,
  'comments': [],
  'id': 100,
  'ipeenscore': 0,
  'lat': 24.984029968789226,
  'lng': 121.95639507263282,
  'name': '擁舞湛藍',
  'reviewdate': '2006/12/13',
  'smallstyle': '咖啡專賣',
  'status': '正常營業',
  'tele': '039780920',
  'url': 'http://www.ipeen.com.tw/shop/100',
  'viewcount': 4461},
 {'Ncomment': 0,
  'address': '台北市松山區敦化北路244巷34號',
  'averagecost': 0,
  'bigstyle': '鍋類',
  'collecount': 0,
  'comments': [],
  'id': 10000,
  'ipeenscore': 0,
  'lat': 25.060937124634002,
  'lng': 121.547771995322